In [1]:
import pandas as pd
from scipy import stats
from scipy.stats import pearsonr
from scipy import stats

In [36]:
df_Comment_mx = pd.read_csv('./comments_sentiment_score_mx.csv')
df_Comment_py = pd.read_csv('./comments_sentiment_score_py.csv')
df_Comment_tf = pd.read_csv('./comments_sentiment_score_tf.csv')

In [37]:
print(df_Comment_mx.shape[0], df_Comment_py.shape[0], df_Comment_tf.shape[0])

1113 7166 2348


In [38]:
df_Comment_mx['SentimentScore'] = pd.to_numeric(df_Comment_mx['SentimentScore'], errors='coerce')
df_Comment_py['SentimentScore'] = pd.to_numeric(df_Comment_py['SentimentScore'], errors='coerce')
df_Comment_tf['SentimentScore'] = pd.to_numeric(df_Comment_tf['SentimentScore'], errors='coerce')

In [39]:
comment_score_mx = df_Comment_mx['SentimentScore'].value_counts()
comment_score_py = df_Comment_py['SentimentScore'].value_counts()
comment_score_tf = df_Comment_tf['SentimentScore'].value_counts()
print(comment_score_mx, comment_score_py, comment_score_tf)

SentimentScore
 0    628
 2    114
 3     99
 1     81
-2     72
-1     52
-3     33
 5     24
-4      6
 4      4
Name: count, dtype: int64 SentimentScore
 0    4070
 2     636
 3     573
 1     562
-2     523
-1     458
 5     144
-3     136
 4      51
-4      13
Name: count, dtype: int64 SentimentScore
 0    1319
 3     291
 2     229
 1     156
-2     132
-1     122
 5      42
-3      27
 4      26
-4       4
Name: count, dtype: int64


In [40]:
df_bugSentiAnalysis_comment_mx = df_Comment_mx.groupby('IssueID').agg(
    RelatedPrID=('RelatedPrID', 'first'),               # 提取第一个 RelatedPrID
    IssueCategory=('IssueCategory', 'first'),
    Participants=('Creator', 'nunique'),                # 计算不同 Creator 的数量
    IsMPLF=('IsMPLF', 'first'),                          # 提取第一个 IsMPLF
    SentimentScore=('SentimentScore', 'mean')           # 计算 SentimentScore 的平均值
).reset_index()
df_bugSentiAnalysis_comment_mx['IssueCategory'] = df_bugSentiAnalysis_comment_mx['IssueCategory'].str.strip()
df_bugSentiAnalysis_comment_mx.to_csv('./bugs_sentiment_score_mx.csv', index=False)

In [41]:
df_bugSentiAnalysis_comment_py = df_Comment_py.groupby('IssueID').agg(
    RelatedPrID=('RelatedPrID', 'first'),               # 提取第一个 RelatedPrID
    IssueCategory=('IssueCategory', 'first'),
    Participants=('Creator', 'nunique'),                # 计算不同 Creator 的数量
    IsMPLF=('IsMPLF', 'first'),                          # 提取第一个 IsMPLF
    SentimentScore=('SentimentScore', 'mean')           # 计算 SentimentScore 的平均值
).reset_index()
df_bugSentiAnalysis_comment_py['IssueCategory'] = df_bugSentiAnalysis_comment_py['IssueCategory'].str.strip()
df_bugSentiAnalysis_comment_py.to_csv('./bugs_sentiment_score_py.csv', index=False)

In [42]:
df_bugSentiAnalysis_comment_tf = df_Comment_tf.groupby('IssueID').agg(
    RelatedPrID=('RelatedPrID', 'first'),               # 提取第一个 RelatedPrID
    IssueCategory=('IssueCategory', 'first'),
    Participants=('Creator', 'nunique'),                # 计算不同 Creator 的数量
    IsMPLF=('IsMPLF', 'first'),                          # 提取第一个 IsMPLF
    SentimentScore=('SentimentScore', 'mean')           # 计算 SentimentScore 的平均值
).reset_index()
df_bugSentiAnalysis_comment_tf['IssueCategory'] = df_bugSentiAnalysis_comment_tf['IssueCategory'].str.strip()
df_bugSentiAnalysis_comment_tf.to_csv('./bugs_sentiment_score_tf.csv', index=False)

In [43]:
Category_sentiment_comment_mx = df_bugSentiAnalysis_comment_mx.groupby('IssueCategory').agg(Count=('IssueID', 'size'), Percentage=('IssueID', lambda x: (len(x) / len(df_bugSentiAnalysis_comment_mx)) * 100), SentimentScore=('SentimentScore', 'mean')).reset_index()
Category_sentiment_comment_py = df_bugSentiAnalysis_comment_py.groupby('IssueCategory').agg(Count=('IssueID', 'size'), Percentage=('IssueID', lambda x: (len(x) / len(df_bugSentiAnalysis_comment_py)) * 100), SentimentScore=('SentimentScore', 'mean')).reset_index()
Category_sentiment_comment_tf = df_bugSentiAnalysis_comment_tf.groupby('IssueCategory').agg(Count=('IssueID', 'size'), Percentage=('IssueID', lambda x: (len(x) / len(df_bugSentiAnalysis_comment_tf)) * 100), SentimentScore=('SentimentScore', 'mean')).reset_index()

Category_sentiment_comment_mx.columns = ['IssueCategory', 'Count', 'Percentage', 'SentimentScore']
Category_sentiment_comment_py.columns = ['IssueCategory', 'Count', 'Percentage', 'SentimentScore']
Category_sentiment_comment_tf.columns = ['IssueCategory', 'Count', 'Percentage', 'SentimentScore']

Category_sentiment_comment_mx.to_excel('./Category_sentiment_comment_mx.xlsx', index=False)
Category_sentiment_comment_py.to_excel('./Category_sentiment_comment_py.xlsx', index=False)
Category_sentiment_comment_tf.to_excel('./Category_sentiment_comment_tf.xlsx', index=False)

In [44]:
def analyze_sentiment_difference(df_MWU, sentiment_col='SentimentScore', group_col='IsMPLF', alpha=0.05):
    # 根据 group_col 列分组
    group_0 = df_MWU[df_MWU[group_col] == 0][sentiment_col]  # 单语言
    group_1 = df_MWU[df_MWU[group_col] == 1][sentiment_col]  # 多语言

    # 正态性检验
    shapiro_0 = stats.shapiro(group_0)
    shapiro_1 = stats.shapiro(group_1)

    print(f'Group 0 (Single Language) Shapiro test: Statistic={shapiro_0.statistic}, p-value={shapiro_0.pvalue}')
    print(f'Group 1 (Multi Language) Shapiro test: Statistic={shapiro_1.statistic}, p-value={shapiro_1.pvalue}')

    # 选择合适的检验
    if shapiro_0.pvalue > 0.05 and shapiro_1.pvalue > 0.05:
        # 数据符合正态分布，可以使用 t 检验
        t_stat, p_value = stats.ttest_ind(group_0, group_1)
        print(f't-test statistic: {t_stat}, p-value: {p_value}')
    else:
        # 数据不符合正态分布，使用 Mann-Whitney U 检验
        u_stat, p_value = stats.mannwhitneyu(group_0, group_1)
        print(f'Mann-Whitney U statistic: {u_stat}, p-value: {p_value}')

    # 根据 p-value 判断显著性
    if p_value < alpha:
        print("Reject null hypothesis: There is a significant difference in SentimentScore between single language and multi-language cases.")
    else:
        print("Fail to reject null hypothesis: No significant difference in SentimentScore between single language and multi-language cases.")

In [45]:
df_total_comment = pd.concat([df_bugSentiAnalysis_comment_mx, df_bugSentiAnalysis_comment_py, df_bugSentiAnalysis_comment_tf]).reset_index(drop=True)

analyze_sentiment_difference(df_total_comment)

Group 0 (Single Language) Shapiro test: Statistic=0.9339575523605297, p-value=1.4746959184926478e-19
Group 1 (Multi Language) Shapiro test: Statistic=0.9135063895621505, p-value=5.278928171609272e-16
Mann-Whitney U statistic: 237636.0, p-value: 0.007821043919907653
Reject null hypothesis: There is a significant difference in SentimentScore between single language and multi-language cases.


In [47]:
average_sentiment = df_total_comment.groupby('IsMPLF')['SentimentScore'].mean()

In [48]:
average_sentiment

IsMPLF
0    0.593843
1    0.451524
Name: SentimentScore, dtype: float64